In [1]:
import numpy as np
import pandas as pd
import os
import tifffile
import torch 
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
from PIL import Image
from utils.data_handlers import *
from utils.models import *
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# Example usage
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder

In [2]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU1.


In [2]:
train_labels_1, train_artifacts_1= link_label1_to_image('train\\image', 'train\\label2')


In [3]:
all_labels_0, all_artifacts_0=link_label0_to_image('No_Artifacts','No_Artifacts_label')



In [4]:
keys = list(all_labels_0.keys())


# Split the keys into training and testing sets
keys_train, keys_test = train_test_split(keys, test_size=0.2, random_state=42)
train_labels_0 = {key: all_labels_0[key] for key in keys_train}
train_artifacts_0 = {key: all_artifacts_0[key] for key in keys_train}
test_labels_0 = {key: all_labels_0[key] for key in keys_test}
test_artifacts_0 = {key: all_artifacts_0[key] for key in keys_test}


In [5]:
train_labels_1.update(train_labels_0)
train_labels=train_labels_1
train_artifacts_0.update(train_artifacts_1)
train_artifacts=train_artifacts_0


In [6]:
print(len(train_labels),len(train_artifacts))

347 347


In [7]:
test_labels_1, test_artifacts_1=link_label1_to_image('test//image', 'test//label2')
print(len(test_labels_1), len(test_artifacts_1))

296 296


In [8]:
test_labels_1.update(test_labels_0)
test_labels=test_labels_1
test_artifacts_0.update(test_artifacts_1)
test_artifacts=test_artifacts_0
print(len(test_labels),len(test_artifacts))

347 347


In [9]:
keys = list(train_labels.keys())


# Split the keys into training and testing sets

keys_train, keys_val = train_test_split(keys, test_size=0.2, random_state=42)
train_labels_final = {key: train_labels[key] for key in keys_train}
train_artifacts_final = {key: train_artifacts[key] for key in keys_train}
val_labels = {key: train_labels[key] for key in keys_val}
val_artifacts = {key: train_artifacts[key] for key in keys_val}

In [10]:
len(val_labels)

70

In [10]:
output_folder = 'train'
train_dict=save_cropped_images_and_labels(train_artifacts_final, output_folder)

In [13]:
len(train_dict)

2493

In [11]:
output_folder = 'val'
val_dict=save_cropped_images_and_labels(val_artifacts, output_folder)

In [15]:
len(val_dict)

630

In [12]:
output_folder = 'test'
test_dict=save_cropped_images_and_labels(test_artifacts, output_folder)

In [15]:
len(test_dict)

3123

In [13]:


train_data = CustomDataset(train_dict)
test_data = CustomDataset(test_dict)
val_data = CustomDataset(val_dict)


In [14]:

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)


In [15]:
model = UNet(in_channels=1, out_channels=2)  # Assuming input grayscale images and 2 classes (background and artifacts)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
train_model(model, train_loader, val_loader, criterion, optimizer,num_epochs=25,model_save_path='best_train_model.pth')
model_path='best_train_model.pth'
test_model(model_path, test_loader, criterion, optimizer, output_folder='test_output')
